In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [7]:
ls

 test/                           train/                          'Untitled1img (1).ipynb'
 test-20240528T231138Z-001.zip   train-20240528T231137Z-001.zip


In [6]:
import zipfile
import os

# Google Drive에 있는 ZIP 파일 경로와 압축 해제할 폴더 경로 설정
zip_file_path = 'test-20240528T231138Z-001.zip'  # ZIP 파일 경로
extract_to_path = './'  # 압축 해제할 폴더 경로

# 폴더가 없으면 생성
if not os.path.exists(extract_to_path):
    os.makedirs(extract_to_path)

# ZIP 파일 열기 및 압축 해제
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
    print(f"압축 해제 완료: {extract_to_path}")
except zipfile.BadZipFile:
    print("Error: The file is not a zip file or it is corrupted.")
except FileNotFoundError:
    print(f"Error: The file {zip_file_path} does not exist.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

압축 해제 완료: ./


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

# 경로 설정
train_dir = './train'
test_dir = './test'

# 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # 데이터 정규화 및 검증 데이터 분할
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 프리트레인드 EfficientNet 모델 불러오기
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# 모델 정의
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# 성능 지표 계산 및 출력
val_preds = model.predict(validation_generator)
val_labels = validation_generator.classes

# MSE 및 RMSE 계산
mse = mean_squared_error(val_labels, np.argmax(val_preds, axis=1))
rmse = np.sqrt(mse)
mae = mean_absolute_error(val_labels, np.argmax(val_preds, axis=1))

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')

# 모델 저장
model.save('efficientnet_image_classifier_model.keras', save_format='keras')

# 모델 불러오기
model = load_model('efficientnet_image_classifier_model.keras')

# 이미지 파일 목록 가져오기
file_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.jpg') or fname.endswith('.png')]

# 데이터프레임 생성
test_df = pd.DataFrame({
    'filename': file_paths,
    'class': ['unknown'] * len(file_paths)  # 임의의 클래스 레이블
})

# 이미지 데이터 제너레이터 설정
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# 데이터프레임에서 데이터 로드
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col=None,
    target_size=(32, 32),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# test_generator에 데이터가 있는지 확인
if len(test_generator.filenames) == 0:
    print("Error: No images found in the test directory.")
else:
    print(f"Found {len(test_generator.filenames)} images in the test directory.")

# 테스트 이미지 예측
test_preds = model.predict(test_generator)
test_pred_labels = np.argmax(test_preds, axis=1)

# 클래스 인덱스를 클래스 이름으로 매핑
class_indices = {v: k for k, v in train_generator.class_indices.items()}
test_pred_class_labels = [class_indices[label] for label in test_pred_labels]

# 결과 저장
results = pd.DataFrame({
    'filename': test_generator.filenames,
    'predicted_label': test_pred_class_labels
})
results.to_csv('classification_results.csv', index=False)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Epoch 1/10
75/75 [==============================] - 55s 192ms/step - loss: 0.9949 - accuracy: 0.5079 - val_loss: 1.1096 - val_accuracy: 0.3351
Epoch 2/10
75/75 [==============================] - 13s 171ms/step - loss: 0.7582 - accuracy: 0.6746 - val_loss: 1.1507 - val_accuracy: 0.3333
Epoch 3/10
75/75 [==============================] - 13s 168ms/step - loss: 0.6631 - accuracy: 0.7262 - val_loss: 1.1802 - val_accuracy: 0.3316
Epoch 4/10
75/75 [==============================] - 11s 149ms/step - loss: 0.5662 - accuracy: 0.7688 - val_loss: 1.2628 - val_accuracy: 0.3403
Epoch 5/10
75/75 [==============================] - 10s 137ms/step - loss: 0.5035 - accuracy: 0.7975 - val_loss: 1.4714 - val_accuracy: 0.3368
Epoch 6/10
75/75 [==============================] - 13s 171ms/step - loss: 0.4792 - accuracy: 0.8092 - val_loss: 1.7041 - val_accuracy: 0.3611
Epoch 7/10
75/75 [==============================] - 12s 157

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

# 경로 설정
train_dir = './train'
test_dir = './test'

# 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # 데이터 정규화 및 검증 데이터 분할
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # 이미지를 224x224로 리사이즈
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # 이미지를 224x224로 리사이즈
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 프리트레인드 ResNet-152 모델 불러오기
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 정의
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=10
)

# 성능 지표 계산 및 출력
val_preds = model.predict(validation_generator)
val_labels = validation_generator.classes

# MSE 및 RMSE 계산
mse = mean_squared_error(val_labels, np.argmax(val_preds, axis=1))
rmse = np.sqrt(mse)
mae = mean_absolute_error(val_labels, np.argmax(val_preds, axis=1))

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')

# 모델 저장
model.save('resnet152_image_classifier_model.keras', save_format='keras')

# 모델 불러오기
model = load_model('resnet152_image_classifier_model.keras')

# 이미지 파일 목록 가져오기
file_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.jpg') or fname.endswith('.png')]

# 데이터프레임 생성
test_df = pd.DataFrame({
    'filename': file_paths,
    'class': ['unknown'] * len(file_paths)  # 임의의 클래스 레이블
})

# 이미지 데이터 제너레이터 설정
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# 데이터프레임에서 데이터 로드
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col=None,
    target_size=(224, 224),  # 이미지를 224x224로 리사이즈
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# test_generator에 데이터가 있는지 확인
if len(test_generator.filenames) == 0:
    print("Error: No images found in the test directory.")
else:
    print(f"Found {len(test_generator.filenames)} images in the test directory.")

# 테스트 이미지 예측
test_preds = model.predict(test_generator)
test_pred_labels = np.argmax(test_preds, axis=1)

# 클래스 인덱스를 클래스 이름으로 매핑
class_indices = {v: k for k, v in train_generator.class_indices.items()}
test_pred_class_labels = [class_indices[label] for label in test_pred_labels]

# 결과 저장
results = pd.DataFrame({
    'filename': test_generator.filenames,
    'predicted_label': test_pred_class_labels
})
results.to_csv('classification_results.csv', index=False)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
234698864/234698864 [==============================] - 7s 0us/step
Epoch 1/10
75/75 [==============================] - 196s 992ms/step - loss: 0.2592 - accuracy: 0.8942 - val_loss: 1.7332 - val_accuracy: 0.3333
Epoch 2/10
75/75 [==============================] - 74s 980ms/step - loss: 0.0463 - accuracy: 0.9875 - val_loss: 1.8670 - val_accuracy: 0.3333
Epoch 3/10
75/75 [==============================] - 74s 986ms/step - loss: 0.0096 - accuracy: 0.9971 - val_loss: 2.2874 - val_accuracy: 0.3383
Epoch 4/10
75/75 [==============================] - 73s 980ms/step - loss: 0.0117 - accuracy: 0.9950 - val_loss: 2.9678 - val_accuracy: 0.3333
Epoch 5/10
75/75 [==============================] - 73s 976ms/step - loss: 0.0466 - accuracy: 0.9833 - val_loss: 1.6819 - val_accuracy: 0.3333
Epoch 6/10
75/75 [==============================] - 74s 980ms/step - loss: 0.0385 - accuracy: 0.9862 - val_loss: 1.6063 - val_accuracy

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

# 경로 설정
train_dir = './train'
test_dir = './test'

# 사용자 정의 RMSE 함수 정의
def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

# 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # 데이터 정규화 및 검증 데이터 분할
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'mse', root_mean_squared_error])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# 검증 데이터 예측
val_preds = model.predict(validation_generator)
val_labels = validation_generator.classes

# MSE 및 RMSE 계산
mse = mean_squared_error(val_labels, np.argmax(val_preds, axis=1))
rmse = np.sqrt(mse)
mae = mean_absolute_error(val_labels, np.argmax(val_preds, axis=1))

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')

model.save('image_classifier_model.h5')

# 모델 불러오기
model = load_model('image_classifier_model.h5')

# 이미지 파일 목록 가져오기
file_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.jpg') or fname.endswith('.png')]

# 데이터프레임 생성
test_df = pd.DataFrame({
    'filename': file_paths,
    'class': ['unknown'] * len(file_paths)  # 임의의 클래스 레이블
})

# 이미지 데이터 제너레이터 설정
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# 데이터프레임에서 데이터 로드
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col=None,
    target_size=(224, 224),  # 이미지를 224x224로 리사이즈
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# test_generator에 데이터가 있는지 확인
if len(test_generator.filenames) == 0:
    print("Error: No images found in the test directory.")
else:
    print(f"Found {len(test_generator.filenames)} images in the test directory.")
    # 테스트 이미지 예측
    test_preds = model.predict(test_generator)
    test_pred_labels = np.argmax(test_preds, axis=1)

    # 클래스 인덱스를 클래스 이름으로 변환
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    test_pred_class_names = [class_indices[label] for label in test_pred_labels]

    # 결과 저장
    results = pd.DataFrame({
        'filename': test_generator.filenames,
        'predicted_label': test_pred_class_names
    })
    results.to_csv('classification_results.csv', index=False)


Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Epoch 1/10
75/75 [==============================] - 20s 176ms/step - loss: 1.0924 - accuracy: 0.5833 - val_loss: 0.7377 - val_accuracy: 0.6823
Epoch 2/10
75/75 [==============================] - 12s 164ms/step - loss: 0.5794 - accuracy: 0.7617 - val_loss: 0.5181 - val_accuracy: 0.7969
Epoch 3/10
75/75 [==============================] - 13s 170ms/step - loss: 0.4324 - accuracy: 0.8246 - val_loss: 0.5358 - val_accuracy: 0.7743
Epoch 4/10
75/75 [==============================] - 13s 171ms/step - loss: 0.3241 - accuracy: 0.8775 - val_loss: 0.5040 - val_accuracy: 0.8108
Epoch 5/10
75/75 [==============================] - 13s 169ms/step - loss: 0.2214 - accuracy: 0.9192 - val_loss: 0.5410 - val_accuracy: 0.8003
Epoch 6/10
75/75 [==============================] - 12s 163ms/step - loss: 0.1581 - accuracy: 0.9413 - val_loss: 0.7256 - val_accuracy: 0.7743
Epoch 7/10
75/75 [==============================] - 12s 164

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 45 validated image filenames.
Found 45 images in the test directory.
45/45 [==============================] - 0s 4ms/step


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

# 경로 설정
train_dir = './train'
test_dir = './test'

# 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)  # 데이터 정규화 및 데이터 증강

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20  # 에포크 수 증가
)

# 검증 데이터 예측
val_preds = model.predict(validation_generator)
val_labels = validation_generator.classes

# MSE 및 RMSE 계산
mse = mean_squared_error(val_labels, np.argmax(val_preds, axis=1))
rmse = np.sqrt(mse)
mae = mean_absolute_error(val_labels, np.argmax(val_preds, axis=1))

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')

model.save('image_classifier_model.h5')

# 모델 불러오기
model = load_model('image_classifier_model.h5')

# 이미지 파일 목록 가져오기
file_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.jpg') or fname.endswith('.png')]

# 데이터프레임 생성
test_df = pd.DataFrame({
    'filename': file_paths,
    'class': ['unknown'] * len(file_paths)  # 임의의 클래스 레이블
})

# 이미지 데이터 제너레이터 설정
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# 데이터프레임에서 데이터 로드
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col=None,
    target_size=(224, 224),  # 이미지를 224x224로 리사이즈
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# test_generator에 데이터가 있는지 확인
if len(test_generator.filenames) == 0:
    print("Error: No images found in the test directory.")
else:
    print(f"Found {len(test_generator.filenames)} images in the test directory.")
    # 테스트 이미지 예측
    test_preds = model.predict(test_generator)
    test_pred_labels = np.argmax(test_preds, axis=1)

    # 클래스 인덱스를 클래스 이름으로 변환
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    test_pred_class_names = [class_indices[label] for label in test_pred_labels]

    # 결과 저장
    results = pd.DataFrame({
        'filename': test_generator.filenames,
        'predicted_label': test_pred_class_names
    })
    results.to_csv('classification_results.csv', index=False)


Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Epoch 1/20
75/75 [==============================] - 52s 662ms/step - loss: 1.5024 - accuracy: 0.4750 - val_loss: 0.8784 - val_accuracy: 0.6128
Epoch 2/20
75/75 [==============================] - 51s 675ms/step - loss: 0.8917 - accuracy: 0.6346 - val_loss: 0.7772 - val_accuracy: 0.6979
Epoch 3/20
75/75 [==============================] - 50s 664ms/step - loss: 0.7689 - accuracy: 0.6812 - val_loss: 0.7884 - val_accuracy: 0.7049
Epoch 4/20
75/75 [==============================] - 53s 703ms/step - loss: 0.7015 - accuracy: 0.7158 - val_loss: 0.7091 - val_accuracy: 0.7413
Epoch 5/20
75/75 [==============================] - 51s 685ms/step - loss: 0.6867 - accuracy: 0.7179 - val_loss: 0.6618 - val_accuracy: 0.7188
Epoch 6/20
75/75 [==============================] - 52s 700ms/step - loss: 0.6581 - accuracy: 0.7262 - val_loss: 0.7169 - val_accuracy: 0.7448
Epoch 7/20
75/75 [==============================] - 51s 681

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 45 validated image filenames.
Found 45 images in the test directory.
45/45 [==============================] - 0s 8ms/step


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, Callback
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

# 사용자 정의 콜백 정의
class MetricsCallback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data
        self.val_labels = []
        for _, y in validation_data:
            self.val_labels.extend(np.argmax(y, axis=1))
        self.val_labels = np.array(self.val_labels)

    def on_epoch_end(self, epoch, logs=None):
        val_preds = self.model.predict(self.validation_data)
        mae = mean_absolute_error(self.val_labels, np.argmax(val_preds, axis=1))
        print(f'Epoch {epoch + 1} - val_mae: {mae:.4f}')

# 경로 설정
train_dir = './train'
test_dir = './test'

# 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)  # 데이터 정규화 및 데이터 증강

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    seed=42
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    seed=42
)

# 프리트레인드 ResNet-152 모델 불러오기
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 일부 레이어 고정
for layer in base_model.layers[:-10]:
    layer.trainable = False

# 모델 정의
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 학습률 감소 콜백
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=0.00001)

# 사용자 정의 콜백 인스턴스 생성
metrics_callback = MetricsCallback(validation_data=validation_generator)

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=10,
    callbacks=[lr_reduction, metrics_callback]
)

# 성능 지표 계산 및 출력 (학습 후)
val_preds = model.predict(validation_generator, steps=len(validation_generator))
val_labels = []
for _, y in validation_generator:
    val_labels.extend(np.argmax(y, axis=1))
val_labels = np.array(val_labels)

# MAE 계산
mae = mean_absolute_error(val_labels, np.argmax(val_preds, axis=1))
print(f'Final MAE: {mae}')

# 모델 저장
model.save('resnet152_image_classifier_model.keras', save_format='keras')

# 모델 불러오기
model = load_model('resnet152_image_classifier_model.keras')

# 이미지 파일 목록 가져오기
file_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.jpg') or fname.endswith('.png')]

# 데이터프레임 생성
test_df = pd.DataFrame({
    'filename': file_paths,
    'class': ['unknown'] * len(file_paths)  # 임의의 클래스 레이블
})

# 이미지 데이터 제너레이터 설정
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# 데이터프레임에서 데이터 로드
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col=None,
    target_size=(224, 224),  # 이미지를 224x224로 리사이즈
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# test_generator에 데이터가 있는지 확인
if len(test_generator.filenames) == 0:
    print("Error: No images found in the test directory.")
else:
    print(f"Found {len(test_generator.filenames)} images in the test directory.")

# 테스트 이미지 예측
test_preds = model.predict(test_generator)
test_pred_labels = np.argmax(test_preds, axis=1)

# 클래스 인덱스를 클래스 이름으로 매핑
class_indices = {v: k for k, v in train_generator.class_indices.items()}
test_pred_class_labels = [class_indices[label] for label in test_pred_labels]

# 결과 저장
results = pd.DataFrame({
    'filename': test_generator.filenames,
    'predicted_label': test_pred_class_labels
})
results.to_csv('classification_results.csv', index=False)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

# 사용자 정의 RMSE 함수 정의
def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

# 경로 설정
train_dir = './train'
test_dir = './test'

# 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)  # 데이터 정규화 및 데이터 증강

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    seed=42  # 동일한 데이터셋을 유지하기 위해 시드 설정
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    seed=42  # 동일한 데이터셋을 유지하기 위해 시드 설정
)

# 프리트레인드 MobileNetV2 모델 불러오기
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 일부 레이어 고정
for layer in base_model.layers[:-10]:
    layer.trainable = False

# 모델 정의
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy', 'mse', root_mean_squared_error])

# 학습률 감소 콜백
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=0.00001)

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=20,
    callbacks=[lr_reduction]
)

# 검증 데이터 예측
val_preds = model.predict(validation_generator, steps=len(validation_generator))
val_labels = validation_generator.classes

# MSE 및 RMSE 계산
mse = mean_squared_error(val_labels, np.argmax(val_preds, axis=1))
rmse = np.sqrt(mse)
mae = mean_absolute_error(val_labels, np.argmax(val_preds, axis=1))

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')

# 모델 저장
model.save('mobilenetv2_image_classifier_model.keras', save_format='keras')

# 모델 불러오기
model = load_model('mobilenetv2_image_classifier_model.keras', custom_objects={'root_mean_squared_error': root_mean_squared_error})

# 이미지 파일 목록 가져오기
file_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.jpg') or fname.endswith('.png')]

# 데이터프레임 생성
test_df = pd.DataFrame({
    'filename': file_paths,
    'class': ['unknown'] * len(file_paths)  # 임의의 클래스 레이블
})

# 이미지 데이터 제너레이터 설정
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# 데이터프레임에서 데이터 로드
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col=None,
    target_size=(224, 224),  # 이미지를 224x224로 리사이즈
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# test_generator에 데이터가 있는지 확인
if len(test_generator.filenames) == 0:
    print("Error: No images found in the test directory.")
else:
    print(f"Found {len(test_generator.filenames)} images in the test directory.")

# 테스트 이미지 예측
test_preds = model.predict(test_generator)
test_pred_labels = np.argmax(test_preds, axis=1)

# 클래스 인덱스를 클래스 이름으로 매핑
class_indices = {v: k for k, v in train_generator.class_indices.items()}
test_pred_class_labels = [class_indices[label] for label in test_pred_labels]

# 결과 저장
results = pd.DataFrame({
    'filename': test_generator.filenames,
    'predicted_label': test_pred_class_labels
})
results.to_csv('classification_results.csv', index=False)


Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Epoch 1/20
75/75 [==============================] - 56s 651ms/step - loss: 0.9307 - accuracy: 0.5929 - mse: 0.1815 - root_mean_squared_error: 0.4233 - val_loss: 0.5927 - val_accuracy: 0.7500 - val_mse: 0.1163 - val_root_mean_squared_error: 0.3380 - lr: 1.0000e-04
Epoch 2/20
75/75 [==============================] - 49s 654ms/step - loss: 0.6275 - accuracy: 0.7433 - mse: 0.1218 - root_mean_squared_error: 0.3463 - val_loss: 0.6746 - val_accuracy: 0.7317 - val_mse: 0.1291 - val_root_mean_squared_error: 0.3581 - lr: 1.0000e-04


KeyboardInterrupt: 